In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict, deque

df = pd.read_csv("resources/HI-Small_Trans.csv")

df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df = df.sort_values('Timestamp').reset_index(drop=True)

# Rename supaya konsisten & clean
df = df.rename(columns={
    'Account': 'sender',
    'Account.1': 'receiver',
    'Amount Paid': 'amount',
    'From Bank': 'from_bank',
    'To Bank': 'to_bank',
    'Is Laundering': 'label'
})

print(df.shape)


(5078345, 11)


In [ ]:
df['hour'] = df['Timestamp'].dt.hour
df['day_of_week'] = df['Timestamp'].dt.dayofweek
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_night'] = df['hour'].isin([22,23,0,1,2,3,4,5]).astype(int)


In [3]:
last_sender = {}
last_receiver = {}

df['sender_time_gap'] = np.nan
df['receiver_time_gap'] = np.nan

for i, row in enumerate(df.itertuples()):
    ts = row.Timestamp

    if row.sender in last_sender:
        df.at[i, 'sender_time_gap'] = (ts - last_sender[row.sender]).total_seconds()
    last_sender[row.sender] = ts

    if row.receiver in last_receiver:
        df.at[i, 'receiver_time_gap'] = (ts - last_receiver[row.receiver]).total_seconds()
    last_receiver[row.receiver] = ts


In [4]:
from collections import defaultdict

fan_out = defaultdict(set)
fan_in = defaultdict(set)

total_rows = len(df)
log_interval = max(1, total_rows // 10)  # tiap 10%

df['fan_out_count'] = 0
df['fan_in_count'] = 0

print("🔁 Menghitung fan-out & fan-in (progress tiap 10%)...")

for i, row in enumerate(df.itertuples()):
    # hitung (sebelum update → no leakage)
    df.at[i, 'fan_out_count'] = len(fan_out[row.sender])
    df.at[i, 'fan_in_count'] = len(fan_in[row.receiver])

    # update struktur
    fan_out[row.sender].add(row.receiver)
    fan_in[row.receiver].add(row.sender)

    # progress log
    if (i + 1) % log_interval == 0 or i == total_rows - 1:
        percent = (i + 1) / total_rows * 100
        print(f"   Progress: {percent:.0f}% ({i+1:,}/{total_rows:,})")

print("✅ Fan-out & fan-in selesai")


🔁 Menghitung fan-out & fan-in (progress tiap 10%)...
   Progress: 10% (507,834/5,078,345)
   Progress: 20% (1,015,668/5,078,345)
   Progress: 30% (1,523,502/5,078,345)
   Progress: 40% (2,031,336/5,078,345)
   Progress: 50% (2,539,170/5,078,345)
   Progress: 60% (3,047,004/5,078,345)
   Progress: 70% (3,554,838/5,078,345)
   Progress: 80% (4,062,672/5,078,345)
   Progress: 90% (4,570,506/5,078,345)
   Progress: 100% (5,078,340/5,078,345)
   Progress: 100% (5,078,345/5,078,345)
✅ Fan-out & fan-in selesai


In [5]:
from collections import defaultdict
import numpy as np

sender_amounts = defaultdict(list)

total_rows = len(df)
log_interval = max(1, total_rows // 10)  # tiap 10%

df['sender_avg_amount'] = np.nan
df['sender_std_amount'] = np.nan

print("💰 Menghitung sender amount statistics (progress tiap 10%)...")

for i, row in enumerate(df.itertuples()):
    hist = sender_amounts[row.sender]

    # hitung dari histori sebelumnya (no leakage)
    if hist:
        df.at[i, 'sender_avg_amount'] = np.mean(hist)
        df.at[i, 'sender_std_amount'] = np.std(hist)

    # update histori
    sender_amounts[row.sender].append(row.amount)

    # progress log
    if (i + 1) % log_interval == 0 or i == total_rows - 1:
        percent = (i + 1) / total_rows * 100
        print(f"   Progress: {percent:.0f}% ({i+1:,}/{total_rows:,})")

print("✅ Sender amount statistics selesai")

# Z-score (vectorized, cepat)
df['sender_amount_zscore'] = (
    (df['amount'] - df['sender_avg_amount']) /
    (df['sender_std_amount'] + 1e-6)
)


💰 Menghitung sender amount statistics (progress tiap 10%)...
   Progress: 10% (507,834/5,078,345)
   Progress: 20% (1,015,668/5,078,345)
   Progress: 30% (1,523,502/5,078,345)
   Progress: 40% (2,031,336/5,078,345)
   Progress: 50% (2,539,170/5,078,345)
   Progress: 60% (3,047,004/5,078,345)
   Progress: 70% (3,554,838/5,078,345)
   Progress: 80% (4,062,672/5,078,345)
   Progress: 90% (4,570,506/5,078,345)
   Progress: 100% (5,078,340/5,078,345)
   Progress: 100% (5,078,345/5,078,345)
✅ Sender amount statistics selesai


In [6]:
from collections import defaultdict, deque

sender_window = defaultdict(deque)

total_rows = len(df)
log_interval = max(1, total_rows // 10)  # tiap 10%

df['tx_count_sender_1h'] = 0

print("⏱️ Menghitung transaction velocity (sender 1h window)...")

for i, row in enumerate(df.itertuples()):
    q = sender_window[row.sender]
    now = row.Timestamp

    # buang transaksi yang lebih lama dari 1 jam
    while q and (now - q[0]).total_seconds() > 3600:
        q.popleft()

    # jumlah transaksi sebelumnya dalam 1 jam (no leakage)
    df.at[i, 'tx_count_sender_1h'] = len(q)

    # update window
    q.append(now)

    # progress log tiap 10%
    if (i + 1) % log_interval == 0 or i == total_rows - 1:
        percent = (i + 1) / total_rows * 100
        print(f"   Progress: {percent:.0f}% ({i+1:,}/{total_rows:,})")

print("✅ Transaction velocity (1h) selesai")


⏱️ Menghitung transaction velocity (sender 1h window)...
   Progress: 10% (507,834/5,078,345)
   Progress: 20% (1,015,668/5,078,345)
   Progress: 30% (1,523,502/5,078,345)
   Progress: 40% (2,031,336/5,078,345)
   Progress: 50% (2,539,170/5,078,345)
   Progress: 60% (3,047,004/5,078,345)
   Progress: 70% (3,554,838/5,078,345)
   Progress: 80% (4,062,672/5,078,345)
   Progress: 90% (4,570,506/5,078,345)
   Progress: 100% (5,078,340/5,078,345)
   Progress: 100% (5,078,345/5,078,345)
✅ Transaction velocity (1h) selesai


In [7]:
from collections import defaultdict

pair_count = defaultdict(int)

total_rows = len(df)
log_interval = max(1, total_rows // 10)  # tiap 10%

df['sender_receiver_tx_count'] = 0

print("🔗 Menghitung sender–receiver transaction count (progress tiap 10%)...")

for i, row in enumerate(df.itertuples()):
    key = (row.sender, row.receiver)

    # jumlah transaksi pasangan SEBELUM transaksi ini (no leakage)
    df.at[i, 'sender_receiver_tx_count'] = pair_count[key]

    # update histori pasangan
    pair_count[key] += 1

    # progress log
    if (i + 1) % log_interval == 0 or i == total_rows - 1:
        percent = (i + 1) / total_rows * 100
        print(f"   Progress: {percent:.0f}% ({i+1:,}/{total_rows:,})")

print("✅ Sender–receiver transaction count selesai")


🔗 Menghitung sender–receiver transaction count (progress tiap 10%)...
   Progress: 10% (507,834/5,078,345)
   Progress: 20% (1,015,668/5,078,345)
   Progress: 30% (1,523,502/5,078,345)
   Progress: 40% (2,031,336/5,078,345)
   Progress: 50% (2,539,170/5,078,345)
   Progress: 60% (3,047,004/5,078,345)
   Progress: 70% (3,554,838/5,078,345)
   Progress: 80% (4,062,672/5,078,345)
   Progress: 90% (4,570,506/5,078,345)
   Progress: 100% (5,078,340/5,078,345)
   Progress: 100% (5,078,345/5,078,345)
✅ Sender–receiver transaction count selesai


In [8]:
from collections import defaultdict

past_edges = defaultdict(set)

total_rows = len(df)
log_interval = max(1, total_rows // 10)  # tiap 10%

df['is_return_flow'] = 0

print("🔄 Menghitung return flow (cycle proxy) dengan progress tiap 10%...")

for i, row in enumerate(df.itertuples()):
    # cek apakah aliran dana kembali (no leakage)
    if row.sender in past_edges[row.receiver]:
        df.at[i, 'is_return_flow'] = 1

    # update histori edge
    past_edges[row.sender].add(row.receiver)

    # progress log
    if (i + 1) % log_interval == 0 or i == total_rows - 1:
        percent = (i + 1) / total_rows * 100
        print(f"   Progress: {percent:.0f}% ({i+1:,}/{total_rows:,})")

print("✅ Return flow feature selesai")


🔄 Menghitung return flow (cycle proxy) dengan progress tiap 10%...
   Progress: 10% (507,834/5,078,345)
   Progress: 20% (1,015,668/5,078,345)
   Progress: 30% (1,523,502/5,078,345)
   Progress: 40% (2,031,336/5,078,345)
   Progress: 50% (2,539,170/5,078,345)
   Progress: 60% (3,047,004/5,078,345)
   Progress: 70% (3,554,838/5,078,345)
   Progress: 80% (4,062,672/5,078,345)
   Progress: 90% (4,570,506/5,078,345)
   Progress: 100% (5,078,340/5,078,345)
   Progress: 100% (5,078,345/5,078,345)
✅ Return flow feature selesai


In [9]:
df['is_cross_bank'] = (df['from_bank'] != df['to_bank']).astype(int)


In [10]:
feature_cols = [
    'amount',
    'hour','day_of_week','is_weekend','is_night',
    'sender_time_gap','receiver_time_gap',
    'fan_out_count','fan_in_count',
    'sender_avg_amount','sender_std_amount','sender_amount_zscore',
    'tx_count_sender_1h',
    'sender_receiver_tx_count',
    'is_return_flow',
    'is_cross_bank',
    'Receiving Currency','Payment Currency','Payment Format'
]

X = df[feature_cols]
y = df['label']

categorical_features = [
    'Receiving Currency','Payment Currency','Payment Format'
]


In [11]:
n = len(df)
train_end = int(n*0.6)
val_end   = int(n*0.8)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val     = X[train_end:val_end], y[train_end:val_end]
X_test, y_test   = X[val_end:], y[val_end:]


In [12]:
from catboost import CatBoostClassifier, Pool

cat_idx = [X.columns.get_loc(c) for c in categorical_features]

train_pool = Pool(X_train, y_train, cat_features=cat_idx)
val_pool   = Pool(X_val, y_val, cat_features=cat_idx)

model = CatBoostClassifier(
    iterations=400,
    depth=6,
    learning_rate=0.08,
    loss_function='Logloss',
    eval_metric='AUC',
    auto_class_weights='Balanced',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.9613272	best: 0.9613272 (0)	total: 841ms	remaining: 5m 35s
100:	test: 0.9846194	best: 0.9846535 (94)	total: 1m 3s	remaining: 3m 6s
200:	test: 0.9849921	best: 0.9850392 (197)	total: 2m 4s	remaining: 2m 3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9852010306
bestIteration = 226

Shrink model to first 227 iterations.


In [13]:
import pandas as pd

importance = model.get_feature_importance(type='PredictionValuesChange')

imp_df = pd.DataFrame({
    'feature': X.columns,
    'importance': importance
}).sort_values('importance', ascending=False)

imp_df.head(15)


,feature,importance
18,Payment Format,25.633844
13,sender_receiver_tx_count,18.371777
0,amount,11.841889
11,sender_amount_zscore,5.848707
5,sender_time_gap,4.807574
7,fan_out_count,4.706022
9,sender_avg_amount,4.194773
15,is_cross_bank,3.867279
10,sender_std_amount,3.483449
6,receiver_time_gap,3.336148


In [14]:
from sklearn.metrics import confusion_matrix

y_prob = model.predict_proba(X_test)[:,1]
y_pred = (y_prob >= 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred)
cm


array([[982337,  31535],
       [   174,   1623]])

In [15]:
tn, fp, fn, tp = 982337, 31535, 174, 1623

precision = tp / (tp + fp)
recall    = tp / (tp + fn)
f1        = 2 * precision * recall / (precision + recall)

print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-score  : {f1:.4f}")


Precision : 0.0489
Recall    : 0.9032
F1-score  : 0.0929


In [16]:
thresholds = [0.2, 0.3, 0.4, 0.5]

from sklearn.metrics import confusion_matrix

for t in thresholds:
    y_pred_t = (y_prob >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_t).ravel()

    recall = tp / (tp + fn)
    precision = tp / (tp + fp)

    print(f"\nThreshold {t}")
    print(f"  TP={tp:,} | FN={fn:,} | FP={fp:,}")
    print(f"  Recall={recall:.3f} | Precision={precision:.3f}")



Threshold 0.2
  TP=1,662 | FN=135 | FP=50,752
  Recall=0.925 | Precision=0.032

Threshold 0.3
  TP=1,657 | FN=140 | FP=42,219
  Recall=0.922 | Precision=0.038

Threshold 0.4
  TP=1,644 | FN=153 | FP=36,462
  Recall=0.915 | Precision=0.043

Threshold 0.5
  TP=1,623 | FN=174 | FP=31,535
  Recall=0.903 | Precision=0.049


In [17]:
import numpy as np
import pandas as pd

THRESHOLD = 0.4

# Salin fitur test
df_test = X_test.copy()

# Tambahkan label asli
df_test['y_true'] = y_test.values

# Probabilitas laundering
df_test['y_prob'] = y_prob

# Prediksi final (berdasarkan threshold)
df_test['y_pred'] = (df_test['y_prob'] >= THRESHOLD).astype(int)


Kondisi Penelitian Saat Ini

Penelitian ini telah berhasil membangun dan mengevaluasi sebuah model deteksi transaksi pencucian uang (Anti Money Laundering/AML) berbasis CatBoostClassifier dengan pendekatan feature engineering lanjutan dan evaluasi yang disesuaikan dengan karakteristik data AML yang sangat tidak seimbang.

1. Dataset dan Pra-pemrosesan

Dataset yang digunakan merupakan dataset transaksi keuangan berskala besar dengan jumlah lebih dari 5 juta transaksi, yang mencerminkan kondisi realistis sistem AML. Data diurutkan secara kronologis berdasarkan waktu transaksi untuk menjaga konsistensi temporal dan mencegah terjadinya data leakage. Pembagian data dilakukan secara chronological split ke dalam data latih, validasi, dan uji.

2. Feature Engineering

Model tidak hanya menggunakan fitur mentah, tetapi diperkuat dengan advanced feature engineering yang dirancang untuk merepresentasikan pola pencucian uang secara perilaku dan temporal, meliputi:

Fitur temporal (jam transaksi, indikator malam, akhir pekan)

Time gap transaksi (jarak waktu antar transaksi sender dan receiver)

Fan-in dan fan-out untuk menangkap pola penyebaran dan pengumpulan dana

Perilaku nominal transaksi, termasuk rata-rata, deviasi, dan z-score amount per sender

Transaction velocity (jumlah transaksi dalam jendela waktu 1 jam)

Relasi sender–receiver (frekuensi interaksi historis)

Return flow (cycle proxy) untuk mendeteksi indikasi aliran dana berputar

Cross-bank indicator untuk mendeteksi obfuscation lintas institusi

Seluruh fitur dihitung secara inkremental berdasarkan histori masa lalu, sehingga tidak memanfaatkan informasi masa depan.

3. Pelatihan Model

Model CatBoost dilatih menggunakan konfigurasi yang disesuaikan untuk data imbalanced, dengan penyesuaian bobot kelas secara otomatis (auto_class_weights='Balanced') dan evaluasi berbasis AUC selama pelatihan. Mekanisme early stopping digunakan untuk menghentikan pelatihan ketika performa tidak lagi meningkat, sehingga model yang dihasilkan stabil dan tidak overfitting.

Model mencapai nilai AUC sebesar 0,985, yang menunjukkan kemampuan pemisahan kelas normal dan laundering yang sangat baik secara global.

4. Evaluasi Model

Evaluasi model dilakukan menggunakan confusion matrix, precision, recall, dan F1-score, karena metrik akurasi dinilai tidak relevan untuk konteks AML akibat ketidakseimbangan kelas yang ekstrem.

Pada threshold default (0,5), model menunjukkan:

Recall ≈ 90%, yang berarti sebagian besar transaksi laundering berhasil terdeteksi

False Negative relatif rendah, sehingga risiko transaksi pencucian uang yang lolos dapat diminimalkan

Precision rendah, yang mengindikasikan adanya alarm palsu, namun masih dapat diterima dalam konteks sistem screening AML

5. Penyesuaian Threshold

Untuk menyeimbangkan trade-off antara false positive dan false negative, dilakukan eksperimen penyesuaian threshold probabilitas. Hasil menunjukkan bahwa:

Threshold rendah meningkatkan recall tetapi menghasilkan alarm palsu yang sangat banyak

Threshold tinggi menurunkan alarm palsu tetapi meningkatkan risiko transaksi laundering yang lolos

Threshold 0,4 dipilih sebagai titik keseimbangan terbaik, karena mampu mempertahankan recall di atas 91% sambil mengurangi jumlah false positive secara signifikan

Threshold ini dianggap paling realistis untuk penggunaan operasional sebagai sistem deteksi awal AML.

6. Inspeksi Manual pada Level Transaksi

Selain evaluasi kuantitatif, penelitian ini telah melakukan inspeksi manual pada level transaksi dengan menampilkan data per baris berdasarkan kategori prediksi:

True Positive (TP): transaksi laundering yang berhasil terdeteksi

False Negative (FN): transaksi laundering yang lolos dari deteksi

False Positive (FP): transaksi normal yang dianggap laundering

True Negative (TN): transaksi normal yang terklasifikasi dengan benar

Pendekatan ini memungkinkan analisis lebih mendalam terhadap pola kesalahan model dan karakteristik transaksi yang sulit dideteksi. Inspeksi manual ini juga menjadi dasar dalam mengidentifikasi keterbatasan model dan potensi pengembangan di masa depan.

7. Kondisi Akhir Model

Secara keseluruhan, model yang dibangun pada penelitian ini:

Tidak berfokus pada akurasi, tetapi pada kemampuan deteksi transaksi laundering

Memprioritaskan recall dan minimisasi false negative

Cocok digunakan sebagai sistem screening awal AML

Menghasilkan alarm palsu yang masih dapat diterima dan dikendalikan melalui threshold tuning

Konsisten dengan praktik dan literatur AML terkini

Kesimpulan Kondisi Saat Ini

Penelitian telah mencapai tahap model yang stabil, tervalidasi, dan dapat dipertanggungjawabkan secara ilmiah. Evaluasi kuantitatif dan kualitatif menunjukkan bahwa model mampu mendeteksi sebagian besar transaksi pencucian uang dengan trade-off yang realistis, sehingga siap untuk dilanjutkan ke tahap penulisan hasil dan pembahasan akhir (Bab IV) serta perumusan kesimpulan.

In [20]:
thresholds = [0.42, 0.45, 0.48]

for t in thresholds:
    y_pred_t = (y_prob >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_t).ravel()

    recall = tp / (tp + fn)
    precision = tp / (tp + fp)

    print(f"\nThreshold {t}")
    print(f"Recall={recall:.3f} | Precision={precision:.3f} | FP={fp:,}")



Threshold 0.42
Recall=0.913 | Precision=0.044 | FP=35,498

Threshold 0.45
Recall=0.908 | Precision=0.046 | FP=34,112

Threshold 0.48
Recall=0.905 | Precision=0.048 | FP=32,495


In [22]:
df_test['alert_rule_stage'] = (
    (df_test['y_prob'] >= 0.4) &
    (
        (df_test['tx_count_sender_1h'] >= 3) |
        (df_test['fan_out_count'] >= 3) |
        (df_test['is_return_flow'] == 1)
    )
).astype(int)
from sklearn.metrics import confusion_matrix

def eval_strategy(y_true, y_pred, name):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)

    print(f"\n{name}")
    print(f"  TP={tp:,} | FN={fn:,} | FP={fp:,}")
    print(f"  Recall={recall:.3f} | Precision={precision:.3f}")
    
    normal_pattern = (
    (df_test['tx_count_sender_1h'] <= 1) &
    (df_test['fan_out_count'] == 0) &
    (df_test['sender_receiver_tx_count'] > 5)
)

normal_pattern = (
    (df_test['tx_count_sender_1h'] <= 1) &
    (df_test['fan_out_count'] == 0) &
    (df_test['sender_receiver_tx_count'] > 5)
)


df_test['alert_soft_whitelist'] = (
    (df_test['y_prob'] >= 0.4) &
    (~normal_pattern)
).astype(int)


df_test['adaptive_threshold'] = np.where(
    df_test['fan_out_count'] >= 5, 0.35, 0.5
)

df_test['alert_adaptive'] = (
    df_test['y_prob'] >= df_test['adaptive_threshold']
).astype(int)


eval_strategy(y_test, df_test['alert_rule_stage'], "Two-stage rule")
eval_strategy(y_test, df_test['alert_soft_whitelist'], "Soft whitelist")
eval_strategy(y_test, df_test['alert_adaptive'], "Adaptive threshold")





Two-stage rule
  TP=909 | FN=888 | FP=8,170
  Recall=0.506 | Precision=0.100

Soft whitelist
  TP=1,644 | FN=153 | FP=36,462
  Recall=0.915 | Precision=0.043

Adaptive threshold
  TP=1,629 | FN=168 | FP=34,393
  Recall=0.907 | Precision=0.045


In [23]:
df_test['final_alert'] = df_test['alert_adaptive']
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, df_test['final_alert']).ravel()

recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1 = 2 * precision * recall / (precision + recall)

print("FINAL MODEL (Adaptive Threshold)")
print(f"TP={tp:,} | FN={fn:,} | FP={fp:,}")
print(f"Recall={recall:.3f}")
print(f"Precision={precision:.3f}")
print(f"F1-score={f1:.3f}")


FINAL MODEL (Adaptive Threshold)
TP=1,629 | FN=168 | FP=34,393
Recall=0.907
Precision=0.045
F1-score=0.086


In [24]:
import numpy as np

last_tx_sender = {}
time_since_sender = np.full(len(df), np.nan)

print("⏱️ Menghitung time_since_last_tx_sender...")

for i, row in enumerate(df.itertuples(index=False)):
    sender = row.sender
    ts = row.Timestamp

    if sender in last_tx_sender:
        time_since_sender[i] = (ts - last_tx_sender[sender]).total_seconds()

    last_tx_sender[sender] = ts

    if (i + 1) % (len(df) // 10) == 0:
        print(f"  Progress: {(i+1)/len(df)*100:.0f}%")

df['time_since_last_tx_sender'] = time_since_sender

print("✅ time_since_last_tx_sender selesai")


⏱️ Menghitung time_since_last_tx_sender...
  Progress: 10%
  Progress: 20%
  Progress: 30%
  Progress: 40%
  Progress: 50%
  Progress: 60%
  Progress: 70%
  Progress: 80%
  Progress: 90%
  Progress: 100%
✅ time_since_last_tx_sender selesai


In [25]:
# --------------------------------------------------
# Gather–Scatter Proxy
# --------------------------------------------------
df['is_gather_scatter'] = (
    (df['fan_in_count'] >= 2) &
    (df['fan_out_count'] >= 2)
).astype(int)

# --------------------------------------------------
# Transaction Burst (Temporal)
# --------------------------------------------------
df['tx_burst_sender'] = (
    (df['tx_count_sender_1h'] >= 5) &
    (df['time_since_last_tx_sender'] <= 300)  # ≤ 5 menit
).astype(int)

# --------------------------------------------------
# Amount Spike (Z-score)
# --------------------------------------------------
df['is_amount_spike'] = (
    df['sender_amount_zscore'].abs() >= 3
).astype(int)

# --------------------------------------------------
# High-Risk Payment Format
# --------------------------------------------------
high_risk_formats = ['ACH', 'Wire']

df['is_high_risk_format'] = (
    df['Payment Format']
    .astype(str)
    .isin(high_risk_formats)
    .astype(int)
)


# --------------------------------------------------
# Cross-Bank + Burst Interaction
# --------------------------------------------------
df['crossbank_burst'] = (
    (df['is_cross_bank'] == 1) &
    (df['tx_burst_sender'] == 1)
).astype(int)


new_features = [
    'is_gather_scatter',
    'tx_burst_sender',
    'is_amount_spike',
    'is_high_risk_format',
    'crossbank_burst'
]

df[new_features].describe()


,is_gather_scatter,tx_burst_sender,is_amount_spike,is_high_risk_format,crossbank_burst
count,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06
mean,7.402729e-01,9.442387e-02,8.241740e-02,1.521464e-01,9.278259e-02
std,4.384849e-01,2.924175e-01,2.749996e-01,3.591628e-01,2.901276e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [26]:
categorical_features = [
    c for c in categorical_features
    if c in X.columns
]
cat_idx = [X.columns.get_loc(c) for c in categorical_features]



In [27]:

train_pool = Pool(X_train, y_train, cat_features=cat_idx)
val_pool   = Pool(X_val, y_val, cat_features=cat_idx)

model = CatBoostClassifier(
    iterations=400,
    depth=6,
    learning_rate=0.08,
    loss_function='Logloss',
    eval_metric='AUC',
    auto_class_weights='Balanced',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.9613272	best: 0.9613272 (0)	total: 1.24s	remaining: 8m 15s
100:	test: 0.9846194	best: 0.9846535 (94)	total: 1m 16s	remaining: 3m 45s
200:	test: 0.9849921	best: 0.9850392 (197)	total: 2m 22s	remaining: 2m 21s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9852010306
bestIteration = 226

Shrink model to first 227 iterations.


In [28]:
# pastikan ini yang dipakai
y_pred_final = df_test['final_alert'].values
y_true = y_test.values
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred_final)
cm


array([[979479,  34393],
       [   168,   1629]])

In [29]:
from catboost import CatBoostClassifier, Pool

model_tuned = CatBoostClassifier(
    iterations=600,          # beri ruang belajar lebih halus
    depth=6,                 # TETAP (sweet spot)
    learning_rate=0.05,      # lebih kecil → learning lebih stabil
    loss_function='Logloss',
    eval_metric='AUC',
    auto_class_weights='Balanced',
    l2_leaf_reg=5,           # REGULARISASI (penting untuk FP)
    border_count=128,        # kurangi noise numerik
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)


model_tuned.fit(train_pool, eval_set=val_pool, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.9617359	best: 0.9617359 (0)	total: 888ms	remaining: 8m 52s
100:	test: 0.9844065	best: 0.9844065 (100)	total: 1m 18s	remaining: 6m 25s
200:	test: 0.9854811	best: 0.9854822 (199)	total: 2m 36s	remaining: 5m 11s
300:	test: 0.9858959	best: 0.9859380 (298)	total: 3m 42s	remaining: 3m 41s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9860185172
bestIteration = 322

Shrink model to first 323 iterations.


In [30]:
# Probabilitas laundering
y_prob_tuned = model_tuned.predict_proba(X_test)[:, 1]

# Adaptive threshold yang sama seperti sebelumnya
adaptive_threshold = np.where(
    X_test['fan_out_count'] >= 5, 0.35, 0.5
)

y_pred_tuned = (y_prob_tuned >= adaptive_threshold).astype(int)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_tuned).ravel()

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)

print("TUNED MODEL (Adaptive Threshold)")
print(f"TP={tp:,} | FN={fn:,} | FP={fp:,}")
print(f"Recall={recall:.3f}")
print(f"Precision={precision:.3f}")
print(f"F1-score={f1:.3f}")


TUNED MODEL (Adaptive Threshold)
TP=1,637 | FN=160 | FP=32,393
Recall=0.911
Precision=0.048
F1-score=0.091


In [31]:
param_grid_lvl2 = [
    # Fokus regularisasi
    {'learning_rate': 0.03, 'l2_leaf_reg': 15, 'border_count': 64,  'depth': 6},
    {'learning_rate': 0.03, 'l2_leaf_reg': 20, 'border_count': 64,  'depth': 6},
    {'learning_rate': 0.04, 'l2_leaf_reg': 15, 'border_count': 64,  'depth': 6},

    # Sedikit kapasitas ekstra
    {'learning_rate': 0.03, 'l2_leaf_reg': 15, 'border_count': 128, 'depth': 7},
    {'learning_rate': 0.04, 'l2_leaf_reg': 20, 'border_count': 128, 'depth': 7},

    # Kontrol kelas ringan (eksperimen)
    {'learning_rate': 0.03, 'l2_leaf_reg': 20, 'border_count': 64,  'depth': 6, 'class_weight_1': 7},
]
from sklearn.metrics import confusion_matrix

results_lvl2 = []

for i, p in enumerate(param_grid_lvl2, 1):
    print(f"\n🔬 Level-2 Experiment {i}/{len(param_grid_lvl2)}: {p}")

    class_weights = None
    if 'class_weight_1' in p:
        class_weights = {0: 1, 1: p['class_weight_1']}

    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=p['learning_rate'],
        depth=p['depth'],
        l2_leaf_reg=p['l2_leaf_reg'],
        border_count=p['border_count'],
        loss_function='Logloss',
        eval_metric='AUC',
        auto_class_weights=None if class_weights else 'Balanced',
        class_weights=class_weights,
        random_seed=42,
        early_stopping_rounds=50,
        verbose=False
    )

    model.fit(train_pool, eval_set=val_pool)

    y_prob = model.predict_proba(X_test)[:, 1]

    adaptive_threshold = np.where(
        X_test['fan_out_count'] >= 5, 0.35, 0.5
    )

    y_pred = (y_prob >= adaptive_threshold).astype(int)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * precision * recall / (precision + recall)

    print(f"TP={tp:,} | FN={fn:,} | FP={fp:,}")
    print(f"Recall={recall:.3f} | Precision={precision:.3f} | F1={f1:.3f}")

    results_lvl2.append({
        **p,
        'TP': tp, 'FN': fn, 'FP': fp,
        'Recall': recall,
        'Precision': precision,
        'F1': f1
    })

results_lvl2_df = pd.DataFrame(results_lvl2)

candidates = results_lvl2_df[
    (results_lvl2_df['Recall'] >= 0.91) &
    (results_lvl2_df['FN'] <= 160)
].sort_values(by='FP')

candidates



🔬 Level-2 Experiment 1/6: {'learning_rate': 0.03, 'l2_leaf_reg': 15, 'border_count': 64, 'depth': 6}
TP=1,693 | FN=104 | FP=83,728
Recall=0.942 | Precision=0.020 | F1=0.039

🔬 Level-2 Experiment 2/6: {'learning_rate': 0.03, 'l2_leaf_reg': 20, 'border_count': 64, 'depth': 6}
TP=1,706 | FN=91 | FP=86,274
Recall=0.949 | Precision=0.019 | F1=0.038

🔬 Level-2 Experiment 3/6: {'learning_rate': 0.04, 'l2_leaf_reg': 15, 'border_count': 64, 'depth': 6}
TP=1,645 | FN=152 | FP=38,983
Recall=0.915 | Precision=0.040 | F1=0.078

🔬 Level-2 Experiment 4/6: {'learning_rate': 0.03, 'l2_leaf_reg': 15, 'border_count': 128, 'depth': 7}
TP=1,645 | FN=152 | FP=44,696
Recall=0.915 | Precision=0.035 | F1=0.068

🔬 Level-2 Experiment 5/6: {'learning_rate': 0.04, 'l2_leaf_reg': 20, 'border_count': 128, 'depth': 7}
TP=1,643 | FN=154 | FP=42,905
Recall=0.914 | Precision=0.037 | F1=0.071

🔬 Level-2 Experiment 6/6: {'learning_rate': 0.03, 'l2_leaf_reg': 20, 'border_count': 64, 'depth': 6, 'class_weight_1': 7}
TP=1,0

,learning_rate,l2_leaf_reg,border_count,depth,TP,FN,FP,Recall,Precision,F1,class_weight_1
2,0.04,15,64,6,1645,152,38983,0.915415,0.040489,0.077549,NaN
4,0.04,20,128,7,1643,154,42905,0.914302,0.036882,0.070903,NaN
3,0.03,15,128,7,1645,152,44696,0.915415,0.035498,0.068345,NaN
0,0.03,15,64,6,1693,104,83728,0.942126,0.019819,0.038822,NaN
1,0.03,20,64,6,1706,91,86274,0.949360,0.019391,0.038005,NaN


In [32]:
import numpy as np

df_test['adaptive_threshold_v2'] = np.select(
    [
        df_test['fan_out_count'] >= 8,
        df_test['fan_out_count'] >= 4
    ],
    [
        0.32,   # sangat agresif (high-risk)
        0.40    # moderat
    ],
    default=0.55  # ketat (low-risk)
)
y_pred_v2 = (df_test['y_prob'] >= df_test['adaptive_threshold_v2']).astype(int)
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_v2).ravel()

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)

print("ADAPTIVE THRESHOLD v2 (FINAL TEST)")
print(f"TP={tp:,} | FN={fn:,} | FP={fp:,}")
print(f"Recall={recall:.3f}")
print(f"Precision={precision:.3f}")
print(f"F1-score={f1:.3f}")


ADAPTIVE THRESHOLD v2 (FINAL TEST)
TP=1,630 | FN=167 | FP=33,591
Recall=0.907
Precision=0.046
F1-score=0.088


# SKRIPSI

## MERMAID DIAGRAM SKRIPSI ##
```mermaid 
flowchart TD
    A(["**Start**<br/>_Mulai proses penelitian_"])
    B["**Pengumpulan Data Transaksi**<br/>Dataset publik dari Kaggle"]
    C["**Pra-Pemrosesan Data**<br/>Konversi waktu, sorting, penyesuaian data"]
    D["**Feature Engineering Berbasis Pola Transaksi**<br/>Pola waktu, frekuensi, relasi akun"]
    E["**Pembagian Data Kronologis**<br/>60% latih, 20% validasi, 20% uji"]
    F["**Data Latih**<br/>Digunakan untuk melatih model"]
    G["**Data Validasi**<br/>Digunakan untuk pemantauan model"]
    H["**Data Uji**<br/>Digunakan untuk evaluasi akhir"]
    I["**Pelatihan Model CatBoost**<br/>Pembelajaran pola transaksi"]
    J["**Visualisasi & Evaluasi Model**<br/>Confusion matrix, metrik klasifikasi"]
    K["**Model Akhir**<br/>Model terpilih hasil evaluasi"]
    LE(["**End**<br/>_Akhir proses penelitian_"])

    A --> B --> C --> D --> E
    E --> F
    E --> G
    E --> H
    F --> I
    G --> I
    I --> J
    J --> K
    K --> LE